In [5]:
import json
import pandas as pd

In [45]:
with open('../amazon_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.json_normalize(data['data']['deals'], sep='.')
df.head(2)

,deal_id,deal_type,deal_title,deal_photo,deal_state,deal_url,canonical_deal_url,deal_starts_at,deal_ends_at,savings_percentage,deal_badge,type,product_asin,deal_price.amount,deal_price.currency,list_price.amount,list_price.currency,savings_amount.amount,savings_amount.currency
0,16cca669,BEST_DEAL,"TurboTax Deluxe Desktop Edition 2025, Federal ...",https://m.media-amazon.com/images/I/71OcM906ML...,AVAILABLE,https://www.amazon.com/TurboTax-Deluxe-Desktop...,https://www.amazon.com/deal/16cca669,2026-01-19T08:00:00.000Z,2026-02-09T07:59:59.000Z,44,44% off,SINGLE_ITEM,B0FWV56H48,44.99,USD,79.99,USD,35.0,USD
1,16f871f2,BEST_DEAL,Lasko 1500W Ceramic Tower Space Heater with Re...,https://m.media-amazon.com/images/I/71wLH8DIhh...,AVAILABLE,https://www.amazon.com/Lasko-751320-Ceramic-He...,https://www.amazon.com/deal/16f871f2,2026-01-23T20:00:00.000Z,2026-02-01T07:59:59.000Z,31,31% off,SINGLE_ITEM,B000TTSXNI,54.99,USD,79.99,USD,25.0,USD


In [52]:
df_dimensional = df[[
    'product_asin',
    'deal_type',
    'deal_title',
    'deal_state',
    'deal_url',
    'deal_photo'
]]
df_dimensional = df_dimensional.rename(columns={
    'product_asin': 'ASIN do produto',
    'deal_type': 'Tipo',
    'deal_title': 'Título',
    'deal_state': 'Estado',
    'deal_url': 'Link da URL',
    'deal_photo': 'Imagem',
})
df_dimensional.head(6)

,ASIN do produto,Tipo,Título,Estado,Link da URL,Imagem
0,B0FWV56H48,BEST_DEAL,"TurboTax Deluxe Desktop Edition 2025, Federal ...",AVAILABLE,https://www.amazon.com/TurboTax-Deluxe-Desktop...,https://m.media-amazon.com/images/I/71OcM906ML...
1,B000TTSXNI,BEST_DEAL,Lasko 1500W Ceramic Tower Space Heater with Re...,AVAILABLE,https://www.amazon.com/Lasko-751320-Ceramic-He...,https://m.media-amazon.com/images/I/71wLH8DIhh...
2,B000SDKDM4,BEST_DEAL,"Lavazza Super Crema Whole Bean Coffee, Medium ...",AVAILABLE,https://www.amazon.com/Lavazza-Coffee-Medium-E...,https://m.media-amazon.com/images/I/61dSjp-nZv...
3,B0B9XB57XM,BEST_DEAL,"EF ECOFLOW Portable Power Station DELTA 2, 102...",AVAILABLE,https://www.amazon.com/EF-ECOFLOW-Portable-Cha...,https://m.media-amazon.com/images/I/61OTWEJPx-...
4,B002QZ11J6,BEST_DEAL,"Dr Infrared Heater Portable Space Heater, Orig...",AVAILABLE,https://www.amazon.com/Dr-Infrared-Heater-Port...,https://m.media-amazon.com/images/I/91BfDvRKGR...
5,B00HFQQ0VU,BEST_DEAL,Waterpik Aquarius Water Flosser For Teeth Clea...,AVAILABLE,https://www.amazon.com/Waterpik-Aquarius-Profe...,https://m.media-amazon.com/images/I/714Q49TDXl...


In [47]:
df_fato = df[[
    'deal_id',
    'product_asin',
    'deal_price.amount',
    'list_price.amount',
    'savings_amount.amount',
    'deal_price.currency',
    'savings_percentage',
    'deal_starts_at',
    'deal_ends_at'
]]

df_fato = df_fato.rename(columns={
    'deal_id': 'ID',
    'product_asin': 'ASIN do produto',
    'deal_price.amount': 'Preço com desconto',
    'list_price.amount': 'Preço original',
    'savings_amount.amount': 'Valor do desconto',
    'deal_price.currency': 'Moeda',
    'savings_percentage': 'Percentual de desconto',
    'deal_starts_at': 'Início da oferta',
    'deal_ends_at': 'Fim da oferta'
})

numeros_cols = [
    'Preço com desconto',
    'Preço original',
    'Valor do desconto',
    'Percentual de desconto'
]

tempodata_cols = [
    'Início da oferta',
    'Fim da oferta'
]

for col in numeros_cols:
    df_fato[col] = pd.to_numeric(df_fato[col], errors='coerce')

for col in tempodata_cols:
    df_fato[col] = pd.to_datetime(df_fato[col], errors='coerce')

df_fato.head(6)


,ID,ASIN do produto,Preço com desconto,Preço original,Valor do desconto,Moeda,Percentual de desconto,Início da oferta,Fim da oferta
0,16cca669,B0FWV56H48,44.99,79.99,35.00,USD,44,2026-01-19 08:00:00+00:00,2026-02-09 07:59:59+00:00
1,16f871f2,B000TTSXNI,54.99,79.99,25.00,USD,31,2026-01-23 20:00:00+00:00,2026-02-01 07:59:59+00:00
2,24aa77bb,B000SDKDM4,16.61,25.19,8.58,USD,34,2026-01-28 08:00:00+00:00,2026-02-04 07:59:59+00:00
3,a0adb60d,B0B9XB57XM,388.00,479.00,91.00,USD,19,2026-01-19 08:00:00+00:00,2026-02-02 07:59:59+00:00
4,27021e0a,B002QZ11J6,87.70,129.99,42.29,USD,33,2026-01-26 08:00:00+00:00,2026-02-01 08:00:00+00:00
5,9c41e87d,B00HFQQ0VU,59.99,79.99,20.00,USD,25,2026-01-25 08:00:00+00:00,2026-02-15 07:59:59+00:00


In [49]:
# Checar se existem duplicadas
print(df_dimensional.duplicated().sum())
print(df_fato.duplicated().sum())

# Contagem de linhas

print(len(df_dimensional))
print(len(df_fato))

0
0
30
30


In [53]:
# Join

join = df_dimensional.merge(df_fato, on='ASIN do produto', how='left')
join

,ASIN do produto,Tipo,Título,Estado,Link da URL,Imagem,ID,Preço com desconto,Preço original,Valor do desconto,Moeda,Percentual de desconto,Início da oferta,Fim da oferta
0,B0FWV56H48,BEST_DEAL,"TurboTax Deluxe Desktop Edition 2025, Federal ...",AVAILABLE,https://www.amazon.com/TurboTax-Deluxe-Desktop...,https://m.media-amazon.com/images/I/71OcM906ML...,16cca669,44.99,79.99,35.00,USD,44,2026-01-19 08:00:00+00:00,2026-02-09 07:59:59+00:00
1,B000TTSXNI,BEST_DEAL,Lasko 1500W Ceramic Tower Space Heater with Re...,AVAILABLE,https://www.amazon.com/Lasko-751320-Ceramic-He...,https://m.media-amazon.com/images/I/71wLH8DIhh...,16f871f2,54.99,79.99,25.00,USD,31,2026-01-23 20:00:00+00:00,2026-02-01 07:59:59+00:00
2,B000SDKDM4,BEST_DEAL,"Lavazza Super Crema Whole Bean Coffee, Medium ...",AVAILABLE,https://www.amazon.com/Lavazza-Coffee-Medium-E...,https://m.media-amazon.com/images/I/61dSjp-nZv...,24aa77bb,16.61,25.19,8.58,USD,34,2026-01-28 08:00:00+00:00,2026-02-04 07:59:59+00:00
3,B0B9XB57XM,BEST_DEAL,"EF ECOFLOW Portable Power Station DELTA 2, 102...",AVAILABLE,https://www.amazon.com/EF-ECOFLOW-Portable-Cha...,https://m.media-amazon.com/images/I/61OTWEJPx-...,a0adb60d,388.00,479.00,91.00,USD,19,2026-01-19 08:00:00+00:00,2026-02-02 07:59:59+00:00
4,B002QZ11J6,BEST_DEAL,"Dr Infrared Heater Portable Space Heater, Orig...",AVAILABLE,https://www.amazon.com/Dr-Infrared-Heater-Port...,https://m.media-amazon.com/images/I/91BfDvRKGR...,27021e0a,87.70,129.99,42.29,USD,33,2026-01-26 08:00:00+00:00,2026-02-01 08:00:00+00:00
5,B00HFQQ0VU,BEST_DEAL,Waterpik Aquarius Water Flosser For Teeth Clea...,AVAILABLE,https://www.amazon.com/Waterpik-Aquarius-Profe...,https://m.media-amazon.com/images/I/714Q49TDXl...,9c41e87d,59.99,79.99,20.00,USD,25,2026-01-25 08:00:00+00:00,2026-02-15 07:59:59+00:00
6,B0C2C9NHZW,BEST_DEAL,"LEVOIT Top Fill Humidifiers for Bedroom, 2.5L ...",AVAILABLE,https://www.amazon.com/LEVOIT-Humidifiers-Humi...,https://m.media-amazon.com/images/I/71xIgTqSN6...,cb7f7ec1,27.96,39.99,12.03,USD,30,2026-01-04 08:00:00+00:00,2026-02-03 07:59:59+00:00
7,B0DJ8ZL7RX,BEST_DEAL,"Sperax Walking Treadmill Pad with APP, 3 in 1 ...",AVAILABLE,https://www.amazon.com/Sperax-Vibration-Treadm...,https://m.media-amazon.com/images/I/61GvlRo10e...,02f86b07,169.98,229.99,60.01,USD,26,2026-01-23 08:00:00+00:00,2026-02-02 07:59:59+00:00
8,B0FRPW868X,BEST_DEAL,H&R Block Tax Software Deluxe + State 2025 Win...,AVAILABLE,https://www.amazon.com/Block-Software-Deluxe-S...,https://m.media-amazon.com/images/I/611uM-Fzip...,f70825ed,29.99,59.99,30.00,USD,50,2026-01-27 08:00:00+00:00,2026-01-30 07:59:59+00:00
9,B0CT9R7WN5,BEST_DEAL,Queen Size 4 Piece Sheet Set - Comfy Breathabl...,AVAILABLE,https://www.amazon.com/Queen-Size-Piece-Sheet-...,https://m.media-amazon.com/images/I/61sKu10Toy...,a3bb36e6,21.24,29.99,8.75,USD,29,2026-01-27 08:00:00+00:00,2026-02-10 07:59:59+00:00
